In [1]:
import serial
import time

In [2]:
END_CHAR = '>'
MESSAGE_TERMINATOR = '\n'

In [3]:
def read_serial_line(ser):
    """
    Reads data from Serial (from Arduino)

    Args:
        ser(byte): Serial line from Arduino
    Returns:
        (str): The output of the line
    """
    line = ser.readline()
    line = line.decode("utf-8")
    data_line = line
    line_output = data_line.strip()
    return line_output

In [4]:
def comm_with_board(arduino, code_array, timeout, print_stats=False):
    message = ','.join([str(x) for x in code_array]) + END_CHAR + MESSAGE_TERMINATOR
    
    write_start = time.time()
    arduino.write(str(message).encode())
    while arduino.out_waiting > 0:
        time.sleep(0.05)
        print('writing...')
    arduino.reset_output_buffer()
    if print_stats:
        print('write time:',time.time() - write_start)

    read_start = time.time()
    while arduino.in_waiting <= 0 and time.time() - read_start < timeout:
        time.sleep(0.05)
    arduino_response = arduino.readline().decode().strip().split(',')
    arduino.reset_input_buffer()
    if print_stats:
        print('read time:',time.time() - read_start)
        print(arduino_response)

Initialize Serial port

In [5]:
# Initialize serial port
serial_port = '/dev/cu.usbmodem14101'
baud_rate = 9600
arduino = serial.Serial(port=serial_port, baudrate=baud_rate, timeout=2)
arduino.flush()
time.sleep(2)

### Loop message testing

In [6]:
code_array_sequence = [
    [1000, 0, 180],
    [1000, 30, 150],
    [1000, 60, 120],
    [1000, 90, 90],
    [1000, 120, 60],
    [1000, 150, 30],
    [1000, 180, 0]
]
runtime = 20
wait_time = 5
print_stats = False

In [7]:
print('COMMENCING SEQUENCE IN ' + str(wait_time) + ' SECONDS...')
time.sleep(wait_time)
start = time.time()
while time.time() - start < runtime:
    for codes in code_array_sequence:
        comm_with_board(arduino, codes, timeout=3, print_stats=print_stats)
    code_array_sequence.reverse()
    for codes in code_array_sequence:
        comm_with_board(arduino, codes, timeout=3, print_stats=print_stats)
    code_array_sequence.reverse()
    print(time.time() - start)
print('SEQUENCE COMPLETE')

COMMENCING SEQUENCE IN 5 SECONDS...
7.47105598449707
14.920825958251953
22.381567001342773
SEQUENCE COMPLETE


### Long message testing
Improves response time of Arduino

In [ ]:
test_message = '1000,0,0' + END_CHAR + MESSAGE_TERMINATOR
read_timeout = 4

In [ ]:
write_start = time.time()
arduino.write(str(test_message).encode())
while arduino.out_waiting > 0:
    time.sleep(0.05)
    print('writing...')
arduino.reset_output_buffer()
print('write time:',time.time() - write_start)

read_start = time.time()
while arduino.in_waiting <= 0 and time.time() - read_start < read_timeout:
    time.sleep(0.05)
arduino_response = arduino.readline().decode().strip().split(',')
arduino.reset_input_buffer()
print('read time:',time.time() - read_start)
print(arduino_response)